In [21]:
from autotrader import AutoData
import json
import requests
import pandas as pd
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
import time 

#------------------------------------------------------------------------------------
#Download's price data and funding rate data for Mycelium Perp Swaps and Binance, has 
#about 45min run time.
#------------------------------------------------------------------------------------

'''Pulling Mycelium funding rate data to csv.'''
#pull historical funding rate and price data
myc_fr_response = requests.get("https://dev.api.tracer.finance/trs/fundingRates")
myc_fr_data = pd.DataFrame(myc_fr_response.json()['rows'])

#write funding rate data to csv
myc_fr_data.to_csv('raw_data/myc_fr.csv')

print('1')
#------------------------------------------------------------------------------------
'''Calculate times for pulling rest of the mycelium price, binance perp funding rate and price data.'''

#get first and last funding rate data times, used for mycelium price data
start_unix_str = myc_fr_data['timestamp'].iloc[-1]
end_unix_str = myc_fr_data['timestamp'].iloc[0]

start_datetime = pd.to_datetime(start_unix_str, unit = 's').to_pydatetime()
end_datetime = pd.to_datetime(end_unix_str, unit = 's').to_pydatetime()

#------------------------------------------------------------------------------------
'''Pulling mycelium price data to csv.'''

#page size for price data
page_size = 20000

#get the number of data entries and first page of data
price_response = requests.get(f'https://dev.api.tracer.finance/trs/priceUpdates?from={start_unix_str}&to={end_unix_str}&page=1&pageSize={page_size}')
price_entries = price_response.json()['totalRecords'] # number of total price points recorded

myc_price = pd.DataFrame(price_response.json()['rows'])
print('2')
#function used in threads to pull price data
def pull_myc_price(start_unix_str, end_unix_str, page_size, page_no):
    price_response = requests.get(f'https://dev.api.tracer.finance/trs/priceUpdates?from={start_unix_str}&to={end_unix_str}&page={page_no}&pageSize={page_size}')
    price_data = pd.DataFrame(price_response.json()['rows'])
    return(price_data)

print('3')
#create threads
with ThreadPoolExecutor(max_workers=6) as executor:
    price_list=[]
    for i in range(2, (price_entries//page_size)+2):
        # submit job:
        price_list.append(executor.submit(
            pull_myc_price,
            start_unix_str = start_unix_str,
            end_unix_str = end_unix_str,
            page_size = page_size,
            page_no = i
        ))
        time.sleep(25)


print('4')
#interpret results of threading   
for i,val in enumerate(price_list):
    result = val.result()
    #print(result)
    #price_data = pd.DataFrame(result.json()['rows'])
    myc_price = pd.concat([myc_price, result])
    #print(i)

print('5')
#write price data to csv
myc_price.to_csv('raw_data/myc_price.csv')


#------------------------------------------------------------------------------------
'''Pulling binance perp funding rate and price data to csv.'''
'''
#the following are the myc universe assets that have perps on binance, ETH = WETH, BTC = WBTC
binance_universe = ['ETH','BTC','LINK','UNI','FXS','BAL','CRV']

# perps on binance come with USDT or BUSD, so both will be used to compare
perp_quote = ['USDT', 'BUSD']

binance = AutoData(data_source ='ccxt', exchange = 'binanceusdm')
for quote in perp_quote:
    for ticker in binance_universe:
        #pull data into data frames
        try:
            fr_data = binance._ccxt_funding_history(f'{ticker}{quote}', start_time=start_datetime, end_time=end_datetime)
        except:
            continue
        try:
            price_data = binance.fetch(f'{ticker}{quote}', granularity='8h', start_time=start_datetime, end_time=end_datetime)
        except:
            continue
        
        #store data in csv's
        fr_data.to_csv(f'raw_data/binance_fr/{ticker}{quote}.csv')
        price_data.to_csv(f'raw_data/binance_price/{ticker}{quote}.csv')

#------------------------------------------------------------------------------------
'''

1
2
3
4


JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [23]:
for i,val in enumerate(price_list):
    #result = val.result()
    #print(result)
    print(val)

<Future at 0x104e025f0 state=finished returned DataFrame>
<Future at 0x1053c4940 state=finished returned DataFrame>
<Future at 0x159a2c6a0 state=finished returned DataFrame>
<Future at 0x1570f3970 state=finished returned DataFrame>
<Future at 0x1570f2aa0 state=finished returned DataFrame>
<Future at 0x1770eb310 state=finished returned DataFrame>
<Future at 0x154d4a7a0 state=finished returned DataFrame>
<Future at 0x154d4b220 state=finished returned DataFrame>
<Future at 0x177440820 state=finished returned DataFrame>
<Future at 0x177092bf0 state=finished returned DataFrame>
<Future at 0x1770932e0 state=finished returned DataFrame>
<Future at 0x159a1ffa0 state=finished returned DataFrame>
<Future at 0x157320640 state=finished returned DataFrame>
<Future at 0x1054be410 state=finished returned DataFrame>
<Future at 0x1054bfdc0 state=finished returned DataFrame>
<Future at 0x159ac7100 state=finished returned DataFrame>
<Future at 0x159ac7e80 state=finished returned DataFrame>
<Future at 0x1

In [273]:
import pandas as pd
pd.options.display.max_rows = 4000

#------------------------------------------------------------------------------------
#Clean's price data and funding rate data for Mycelium Perp Swaps to 8hr increments
#------------------------------------------------------------------------------------

''''Read Mycelium funding rate data from csv and put in formatable format'''
#read raw data from csv
myc_fr_raw = pd.read_csv('raw_data/myc_fr.csv', 
        usecols=['timestamp','symbol','endFundingRate','startFundingRate']
) 

#define funding rate columns
fr_symbols = myc_fr_raw.symbol
fr_timestamp = pd.to_datetime(myc_fr_raw['timestamp'], unit = 's')
fr = (myc_fr_raw.endFundingRate.astype(int) - myc_fr_data.startFundingRate.astype(int))/1000000

#create Mycelium Funding rate data frame
myc_fr = pd.DataFrame(
        data = {'timestamp':fr_timestamp,
                'symbol' : fr_symbols,
                'funding_rate' :fr}).set_index('timestamp').iloc[::-1]

#last date in records
end = myc_fr.index[-1]

#truncation start date for all data
trunc_start = myc_fr.index[0]

''''Read Mycelium price data from csv and put in formatable format'''

#read raw data from csv 
myc_price_raw = pd.read_csv('raw_data/myc_price.csv',
                            usecols=['timestamp', 'symbol','price'])

#define funding rate columns
price_symbol = myc_price_raw.symbol
price_timestamp = pd.to_datetime(myc_price_raw['timestamp'], unit = 's')
        #numbers 27 digits are too large to calculate in pd.Dataframe
price = myc_price_raw['price'].str.removesuffix('000000000000000000000000000') # removes 27 zero's, divide 10^27
price = price.astype({'price':'int'})/1000

myc_price = pd.DataFrame(
        data = {'timestamp':price_timestamp,
                'symbol' : price_symbol,
                'price' : price}).set_index('timestamp').iloc[::-1]


'''Iterate through funding rate and price data filtering by token, reindexing to constant 
   frequecy datetime and populate missing data entries from reindexing'''

#create list to store data frames for each token
token_data = []
#mycelium universe
myc_universe = ['WETH','WBTC','LINK','UNI','USDC','USDT','DAI','FRAX','FXS','BAL','CRV']

#define truncation start date
for ticker in myc_universe:
        #create dataframe for each ticker
        fr_ = myc_fr[myc_fr.symbol == ticker]
        price_ = myc_price[myc_price.symbol == ticker]

        #set truncation start date
        start_time = fr_.index[0]
        if start_time > trunc_start:
                trunc_start = start_time

        #remove duplicates from the price data
        price_ = price_[~price_.index.duplicated(keep='first')]

        #create indexes for reindexing
        hourly_index = pd.date_range(start= trunc_start, end= end, freq = 'h') # create index w/ const freq for comparison
        sec_index = pd.date_range(start= trunc_start, end=end, freq = 's') # create index to fill entries at the top of each hour, e.g. 
                                                                                # need data at 8:00:00 to reindex back to h

        #reindex data and populate timestamps with missings data
        fr_data = fr_.reindex(index = hourly_index, method = 'ffill')
        price_data = price_.reindex(index = sec_index, method = 'ffill')
        price_data = price_data.reindex(index = hourly_index)

        #create dataframe containing Funding Rate and price data for each token
        data_ = pd.DataFrame(data = {'timestamp':hourly_index,
                                     'funding_rate':fr_data['funding_rate'],
                                     'price':price_data['price']})

        data_dict = {'token': f'{ticker}',
                    'myc_data': data_}

        token_data.append(data_dict)



#print(trunc_start)

#------------------------------------------------------------------------------------
### END OF PRODUCTION ###
#------------------------------------------------------------------------------------

In [18]:
print(start_unix_str)

1664629200


In [282]:
print(price_index)
print(price_)

DatetimeIndex(['2022-10-01 20:00:00', '2022-10-01 20:00:01',
               '2022-10-01 20:00:02', '2022-10-01 20:00:03',
               '2022-10-01 20:00:04', '2022-10-01 20:00:05',
               '2022-10-01 20:00:06', '2022-10-01 20:00:07',
               '2022-10-01 20:00:08', '2022-10-01 20:00:09',
               ...
               '2022-11-11 05:59:51', '2022-11-11 05:59:52',
               '2022-11-11 05:59:53', '2022-11-11 05:59:54',
               '2022-11-11 05:59:55', '2022-11-11 05:59:56',
               '2022-11-11 05:59:57', '2022-11-11 05:59:58',
               '2022-11-11 05:59:59', '2022-11-11 06:00:00'],
              dtype='datetime64[ns]', length=3492001, freq='S')
                    symbol  price
timestamp                        
2022-11-10 00:20:07    CRV  0.558
2022-11-10 00:20:10    CRV  0.557
2022-11-10 00:20:13    CRV  0.558
2022-11-10 00:20:17    CRV  0.558
2022-11-10 00:20:19    CRV  0.557
...                    ...    ...
2022-11-11 04:59:18    CRV  0.649


In [286]:
print(myc_price_raw[myc_price_raw.symbol == 'WBTC'])

       symbol                                price   timestamp
6        WBTC  17060500000000000000000000000000000  1668142783
10       WBTC  17048500000000000000000000000000000  1668142771
19       WBTC  17048500000000000000000000000000000  1668142768
23       WBTC  17048500000000000000000000000000000  1668142765
30       WBTC  17048500000000000000000000000000000  1668142758
...       ...                                  ...         ...
167071   WBTC  16011000000000000000000000000000000  1668039619
167079   WBTC  16011000000000000000000000000000000  1668039617
167084   WBTC  15994105000000000000000000000000000  1668039613
167094   WBTC  15991270000000000000000000000000000  1668039610
167099   WBTC  15991030000000000000000000000000000  1668039607

[23853 rows x 3 columns]


In [ ]:
import pandas as pd
# from here down is either in construction or fucked

#remove token addresses
myc_fr_data = myc_fr_data.drop(['token'], axis =1)

#define funding rate columns
fr = (myc_fr_data.endFundingRate.astype(int) - myc_fr_data.startFundingRate.astype(int))/1000000

time = pd.to_datetime(myc_fr_data['timestamp'], unit = 's')
token = myc_fr_data.symbol

myc_funding = pd.concat([time,token,fr], axis=1)
myc_funding = myc_funding.rename(columns={0:'funding_rate'})

##!!!!###
token_data = []
for ticker in myc_universe:
        data = myc_fr[myc_fr.symbol == ticker]
        data_dict = {
                'token': f'{ticker}',
                'data': data,
        }
        print(data)
        token_data.append(data_dict)

## new block ##
import numpy as np
page_size = 100
price_response = requests.get(f'https://dev.api.tracer.finance/trs/priceUpdates?from={start_unix_str}&to={end_unix_str}&page=1&pageSize{page_size}')
price_entries = price_response.json()['totalRecords']
myc_price = pd.DataFrame(price_response.json()['rows'])

for i in range(2,(price_entries//100)+2):
    price_response = requests.get(f'https://dev.api.tracer.finance/trs/priceUpdates?from={start_unix_str}&to={end_unix_str}&page={i}&pageSize{page_size}')
    price_data = pd.DataFrame(price_response.json()['rows'])
    myc_price = pd.concat([myc_price, price_data])

#remove headers that aren't required
myc_price = myc_price.drop(['token','txnHash','blockNumber'], axis =1)

#refactor prices to USD
myc_price['price'] = myc_price['price'].str.removesuffix('000000000000000000000000000') # removes 27 zero's
myc_price['price'] = myc_price['price'].astype({'price':'int'})/1000


#Change UNIX time string objects to datetime objects
myc_price['timestamp'] = pd.to_datetime(myc_price['timestamp'], unit = 's')
myc_price.to_csv('myc_price_data/price_data.csv')

print(len(myc_price.price))
print(myc_price.to_markdown())

### new block
start_unix_str = myc_fr_data['timestamp'].iloc[-1]
end_unix_str = myc_fr_data['timestamp'].iloc[0]

start_unix = pd.to_datetime(end_unix_str, unit = 's').to_pydatetime()
end_unix = pd.to_datetime(end_unix_str, unit = 's').to_pydatetime()

print(start_unix)
print(type(start_unix))

### new block ###
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import json
import requests

page_size = 1000

price_response = requests.get(f'https://dev.api.tracer.finance/trs/priceUpdates?from={start_unix_str}&to={end_unix_str}&page=1&pageSize{page_size}')
price_entries = price_response.json()['totalRecords']
myc_price = pd.DataFrame(price_response.json()['rows'])

def pull_myc_price(start_unix_str, end_unix_str, page_size, page_no):
    price_response = requests.get(f'https://dev.api.tracer.finance/trs/priceUpdates?from={start_unix_str}&to={end_unix_str}&page={page_no}&pageSize{page_size}')
    price_data = pd.DataFrame(price_response.json()['rows'])
    return(price_data)

with ThreadPoolExecutor(max_workers=6) as executor:
    price_list=[]
    for i in range(2, (price_entries//page_size)+2):
        # submit job:
        price_list.append(executor.submit(
            pull_myc_price,
            start_unix_str = start_unix_str,
            end_unix_str = end_unix_str,
            page_size = page_size,
            page_no = i
        ))
    
for i,val in enumerate(price_list):
    result = price_list[i].result()
    myc_price = pd.concat([myc_price,result])

myc_price.to_csv('raw_data/myc_price.csv')

### new block ###
fr_data = response.json()['rows']
df_data = pd.DataFrame(fr_data)
df_data = df_data.drop(['token','endFundingRate'], axis = 1)
print(df_data.to_markdown())

#for i in fr_data['rows']:
#    print(i)